# Create MEGAN tree and GraphIan visualization

- create csv to export to MEGAN
- create GraphIan layers
- Look at 18S vs COI across both Banzai and Usearch (by Genus)

In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

#For illustrator import:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [2]:
#Functions

#Raw Read Numbers
def make_rawread(infile):
    #infile = OTU_table_taxa_all.txt
    df = pd.read_csv(infile, sep='\t')
    df.rename(columns={'DUP_ID':'OTU'}, inplace=True)
    df.drop('taxonomy', axis=1, inplace=True)
    df.drop('repseq', axis=1, inplace=True)
    df.set_index('OTU', inplace=True)
    return df
    
#metadata handling and sample order
def make_metadata(infile):
    df = pd.read_csv(infile)
    #make all column names with underscore
    columns_l = list(df)
    columns_l = [c.replace('-', '_') for c in columns_l]
    df.columns = columns_l
    #extract site name
    df['site'] = df['sample_name'].str.extract('([a-zA-Z0-9]+)')
    df_full = df[:]
    #add in some missing location data
    df_full.loc[df_full['site']=='UC5', 'dec_lat'] = 32
    df_full.loc[df_full['site']=='UC5', 'dec_long'] = '.118.17'
    df_full['dec_long'] = df_full['dec_long'].str.replace('^\.','-')
    df_full.set_index('sample_name', inplace=True)
    df= df[['Description', 'Treatment', 'sample_name']]
    df['comb'] = df['Description']+'_'+df['Treatment']+'_'+df['sample_name']
    df['site_Order']=df['sample_name'].str.replace('GOC2a', '16').str.replace('GOC2b', '17').str.replace('NTC', '0')
    df['site_Order']=df['site_Order'].str.extract('(\d+)')
    df.set_index('sample_name', inplace=True)
    df['site_Order']= df['site_Order'].astype(int)
    df.sort_values('site_Order', ascending=True, inplace=True)
    return df_full, df

#DESEQ Numbers
def make_deseq(infile):
    df = pd.read_csv(infile, sep=',')
    df.rename(columns={'Unnamed: 0':'OTU'}, inplace=True)
    df.set_index('OTU', inplace=True)
    return df

#Rarefied Read Numbers (From R script); make compositional
def make_rarereads_comp(infile):
    df= pd.read_csv(infile, sep=',')
    df.rename(columns={'Unnamed: 0':'OTU'}, inplace=True)
    df.set_index('OTU', inplace=True)
    df=df.T
    df['Total']=df.sum(axis=1)
    tot_reads = df['Total'].tolist()
    tot_reads = set(tot_reads)
    print('Rarefied read count:', tot_reads)
    tot_reads = df['Total'].tolist()[0]
    df = df/tot_reads *100
    df.drop('Total', axis=1, inplace=True)
    df=df.T
    return df

def make_rarereads(infile):
    df= pd.read_csv(infile, sep=',')
    df.rename(columns={'Unnamed: 0':'OTU'}, inplace=True)
    df.set_index('OTU', inplace=True)
    return df

#Taxa table
def make_taxa(infile):
    #infile = OTU_table_taxa_all.txt
    df = pd.read_csv(infile, sep='\t')
    df.rename(columns={'DUP_ID':'OTU'}, inplace=True)
    df = df[['taxonomy', 'OTU']]
    df.set_index('OTU', inplace=True)
    df['Kingdom']=df['taxonomy'].str.split("\'").str[1]
    df['Phylum']=df['taxonomy'].str.split("\'").str[3]
    df['Class']=df['taxonomy'].str.split("\'").str[5]
    df['Order']=df['taxonomy'].str.split("\'").str[7]
    df['Family']=df['taxonomy'].str.split("\'").str[9]
    df['Genus']=df['taxonomy'].str.split("\'").str[11]
    df['Species']=df['taxonomy'].str.split("\'").str[13]
    df=df.drop('taxonomy', axis=1)
    return df

#Filtered Taxa table
def make_Ftaxa(infile):
    df = pd.read_csv(infile, sep=',')
    df.rename(columns={'OTU_ID':'OTU'}, inplace=True)
    df.set_index('OTU', inplace=True)
    df=df[df.columns[-7:]]
    return df

#extract sequences
def make_seq(infile):
    #infile = OTU_table_taxa_all.txt
    df = pd.read_csv(infile, sep='\t')
    df.rename(columns={'DUP_ID':'OTU'}, inplace=True)
    df.set_index('OTU', inplace=True)
    df=df[['repseq']]
    return df

# Import Banzai Results

In [3]:
#COI M6
print('COI')
#OTU_table_taxa_all.txt location
file_loc1 = '/Users/kpitz/Projects/Gulf_of_California/Cutadapt_Results/COI/Analysis_20190921_1416/all_lib/Post_Blast_20190925_0914/OTU_table_taxa_all.txt'
#metadata
meta_file = '/Users/kpitz/Projects/Gulf_of_California/Cutadapt_Results/COI/Analysis_20190921_1416/GOC_20190921_1416_COI_analysis_metadata.csv'

#DESEQ data
#file2 = '/Users/kpitz/Projects/Gulf_of_California/Deseq/COI_GOC_DEseq_122117.csv'
#rarefied data
file3 = '/Users/kpitz/Projects/MBON/Rarefied_Data_unmerged/GOC_COI_OTU_Table_092619_M6.csv'
#Filtered OTU table
file4 ='/Users/kpitz/Projects/Gulf_of_California/Cutadapt_Results/COI/Analysis_20190921_1416/all_lib/Filtered_OTU_taxa_table_all.csv'

raw_COI = make_rawread(file_loc1)
meta_COI, samp_lim_COI = make_metadata(meta_file)
#deseq_COI = make_deseq(file2)
rare_COI = make_rarereads(file3)
rare_comp_COI = make_rarereads_comp(file3)
taxa_COI = make_taxa(file_loc1)
Ftaxa_COI = make_Ftaxa(file4)
seq_COI = make_seq(file_loc1)

#18S_M6
print('18S')
#OTU_table_taxa_all.txt location
file_loc1 = '/Users/kpitz/Projects/Gulf_of_California/Cutadapt_Results/18S/Analysis_20190924_1129/all_lib/Post_Blast_20190930_1343/OTU_table_taxa_all.txt'
#metadata
meta_file = '/Users/kpitz/Projects/Gulf_of_California/Cutadapt_Results/18S/Analysis_20190924_1129/GOC_18S_Metadata.csv'
#DESEQ data
#file2 = '/Users/kpitz/Projects/Gulf_of_California/Deseq/18S_GOC_DEseq_122117.csv'
#rarefied data
file3 = '/Users/kpitz/Projects/MBON/Rarefied_Data_unmerged/GOC_18S_OTU_Table_100119_M6.csv'
#Filtered OTU table
file4 = '/Users/kpitz/Projects/Gulf_of_California/Cutadapt_Results/18S/Analysis_20190924_1129/all_lib/Filtered_OTU_taxa_table_all.csv'


raw_18S = make_rawread(file_loc1)
meta_18S, samp_lim_18S = make_metadata(meta_file)
#deseq_18S = make_deseq(file2)
rare_18S = make_rarereads(file3)
rare_comp_18S = make_rarereads_comp(file3)
taxa_18S = make_taxa(file_loc1)
Ftaxa_18S = make_Ftaxa(file4)
seq_18S = make_seq(file_loc1)


#Directory for saving Figures
plot_dir = '/Users/kpitz/Projects/Gulf_of_California/GOC_18S_COI_Combined_Cutadapt/'
plot_name = 'GOC_18SCOI_'
Plot_str = plot_dir + plot_name
print(Plot_str)

#Use to change the name of the databases
name1 = 'GOC_18SCOI_'

COI
Rarefied read count: {129363}
18S
Rarefied read count: {28676}
/Users/kpitz/Projects/Gulf_of_California/GOC_18S_COI_Combined_Cutadapt/GOC_18SCOI_


# Import Filtered Banzai Results

Created in GOC_COI_Filter_Data.ipynb

- /Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_COI_seq_table_092519.csv
- /Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_COI_otu_table_092519.csv
- /Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_COI_taxa_table_092519.csv

In [4]:
files = ['/Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_COI_seq_table_092519.csv',
         '/Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_COI_otu_table_092519.csv',
         '/Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_COI_taxa_table_092519.csv']
#dfs = [seq_COI_filt, raw_COI_filt, Ftaxa_COI_filt]
df = pd.read_csv(files[0])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('OTU', inplace=True)
seq_COI_filt = df.copy()

df = pd.read_csv(files[1])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('OTU', inplace=True)
raw_COI_filt = df.copy()

df = pd.read_csv(files[2])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('OTU', inplace=True)
Ftaxa_COI_filt = df.copy()

raw_COI_filt.head()

#import metadata
meta_file ='/Users/kpitz/Projects/Gulf_of_California/Combined_PCTD_Metadata_043019.csv'
df= pd.read_csv(meta_file)
df=df.sort_values(['order'])
df.set_index('sample_ID', inplace=True)
print(list(df))

meta_COI_PCTD = df.copy()
meta_COI_PCTD.head()

['order', 'tag_sequence', 'primer_sequence_F', 'primer_sequence_R', 'library_tag_combo', 'library', 'sample_type', 'locus', 'tag_number', 'R1', 'R2', 'Treatment', 'Time_of_Day', 'Description', 'Description_3', 'site', 'SEQ', 'BOTTLE', 'DEPTH', 'CRUISE', 'PLATFORM', 'DEC_LAT', 'DEC_LONG', 'TMP', 'SAL', 'CHL_GFF', 'PRESSURE', 'NO3', 'OXY_ML', 'RDEP', 'TRANSMISS', 'SIG_T', 'FLUOR', 'DATE_TIME', 'cruise', 'SEQAvg_dg', 'AvgOfTMP', 'StDevOfTMP', 'CountOfTMP', 'AvgOfSAL1', 'StDevOfSAL', 'CountOfSAL', 'AvgOfCHLA', 'StDevOfCHLA', 'CountOfCHLA', 'AvgOfOXY_ML1', 'CountOfOXY_ML1', 'CountOfOXY_ML', 'AvgOfTRANSMISS', 'StDevOfTRANSMISS', 'CountOfTRANSMISS', 'AvgOfSIGMA_THETA', 'StDevOfSIGMA_THETA', 'CountOfSIGMA_THETA']


,order,tag_sequence,primer_sequence_F,primer_sequence_R,library_tag_combo,library,sample_type,locus,tag_number,R1,...,CountOfCHLA,AvgOfOXY_ML1,CountOfOXY_ML1,CountOfOXY_ML,AvgOfTRANSMISS,StDevOfTRANSMISS,CountOfTRANSMISS,AvgOfSIGMA_THETA,StDevOfSIGMA_THETA,CountOfSIGMA_THETA
sample_ID,,,,,,,,,,,,,,,,,,,,,
UC1,1.0,TTCTAGCT,CACGACGTTGTAAAACGAC,GGATAACAATTTCACACAGG,N7_TTCTAGCT,N7,environmental,COI,1,UC1-COI_S22_L001_R1_001.fastq.gz,...,101,4.649444,101,101,88.565879,1.576289,101,25.500296,0.326118,101
UC2,2.0,CCTAGAGT,CACGACGTTGTAAAACGAC,GGATAACAATTTCACACAGG,N8_CCTAGAGT,N8,environmental,COI,2,UC2-COI_S23_L001_R1_001.fastq.gz,...,100,4.906637,100,100,88.278839,1.773990,100,25.359315,0.297979,100
UC3,3.0,GCGTAAGA,CACGACGTTGTAAAACGAC,GGATAACAATTTCACACAGG,N11_GCGTAAGA,N11,environmental,COI,3,UC3-COI_S24_L001_R1_001.fastq.gz,...,100,3.438635,100,100,89.038490,2.019294,100,25.820711,0.395819,100
UC3_1,3.2,GCGTAAGA,CACGACGTTGTAAAACGAC,GGATAACAATTTCACACAGG,N9_GCGTAAGA,N9,environmental,COI,19,UC3-1-COI_S40_L001_R1_001.fastq.gz,...,100,3.438635,100,100,89.038490,2.019294,100,25.820711,0.395819,100
UC3_2,3.3,TTCTAGCT,CACGACGTTGTAAAACGAC,GGATAACAATTTCACACAGG,N10_TTCTAGCT,N10,environmental,COI,17,UC3-2-COI_S38_L001_R1_001.fastq.gz,...,100,3.438635,100,100,89.038490,2.019294,100,25.820711,0.395819,100


In [5]:
files = ['/Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_18S_seq_table_100119.csv',
         '/Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_18S_otu_table_100119.csv',
         '/Users/kpitz/Projects/Gulf_of_California/Decontaminated_tables/GOC_18S_taxa_table_100119.csv']
#dfs = [seq_18S_filt, raw_18S_filt, Ftaxa_18S_filt]
df = pd.read_csv(files[0])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('OTU', inplace=True)
seq_18S_filt = df.copy()

df = pd.read_csv(files[1])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('OTU', inplace=True)
raw_18S_filt = df.copy()

df = pd.read_csv(files[2])
df.set_index('Unnamed: 0', inplace=True)
df.index.rename('OTU', inplace=True)
Ftaxa_18S_filt = df.copy()

df.head()

,Kingdom,Phylum,Class,Order,Family,Genus,Species
OTU,,,,,,,
OTU_1,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,s_
OTU_10,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Metridinidae,Metridia,Metridia lucens
OTU_100,Eukaryota,Arthropoda,Malacostraca,Euphausiacea,Euphausiidae,g_,s_
OTU_100012,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Metridinidae,Metridia,Metridia lucens
OTU_100017,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Calanidae,Calanus,s_


# Import USEARCH Data

In [6]:
#rarefied datasets
Geller_18S = pd.read_csv('/Users/kpitz/Projects/Gulf_of_California/Geller_lab_data/18S_Geller_MGmodified_9Jan19.txt', sep='	')
Geller_18S.set_index('#OTU ID', inplace=True)
Geller_18S
#original 18S before mistake fixed (OTU_67 included and taxonomy wrong- sequences also wrong)
#Geller_18So = pd.read_csv('/Users/kpitz/Projects/Gulf_of_California/Geller_lab_data/Incorrect_18S/18S_Geller.txt', sep='	')
#Geller_18So
Geller_COI = pd.read_csv('/Users/kpitz/Projects/Gulf_of_California/Geller_lab_data/COI_Geller.txt', sep='	')
Geller_COI.set_index('#OTU ID', inplace=True)
Geller_COI

#non-rarefied datasets
file= '/Users/kpitz/Projects/Gulf_of_California/Geller_lab_data/Not_rarefied/MBARI_18S_otu_table_10097_txt.txt'
Geller_18S_nr = pd.read_csv(file, sep='	')
Geller_18S_nr.set_index('#OTU ID', inplace=True)
Geller_18S_nr
file= '/Users/kpitz/Projects/Gulf_of_California/Geller_lab_data/Not_rarefied/MBARI_COI_otu_table_10095_txt.txt'
Geller_COI_nr = pd.read_csv(file, sep='	')
Geller_COI_nr.set_index('#OTU ID', inplace=True)
Geller_18S_nr.head()




,CP23_1,CP23,CP23_2,GOC2a,GOC2b,UC10,UC1,UC12,UC13,UC14,UC15,UC2,UC3_1,UC3,UC3_2,UC4,UC5,UC6,UC7,UC9
#OTU ID,,,,,,,,,,,,,,,,,,,,
OTU_6,19231,22170,14287,34957,22575,68966,6533,28481,32333,14665,34219,3970,28208,41001,39997,13079,2023,373,10606,1568
OTU_28,16679,18639,13582,1344,1294,5722,25575,26363,29378,14098,32274,21545,20676,23539,31666,16134,14833,35070,25123,24619
OTU_89,10753,11618,4602,22,0,1691,0,1196,223,1298,725,3,0,0,0,3,0,0,0,1
OTU_34,7817,9214,2665,1670,1346,306,27,2595,1881,2881,2781,1955,189,299,68,19,17,1011,36,40
OTU_168,3404,3930,1326,490,244,0,0,1,0,773,100,0,0,0,0,0,0,0,0,0


In [7]:
#Make compositional rarefied OTU table and taxa table
# Format Geller Data to get OTU table / taxa table
print(list(Ftaxa_COI))
#levels = list(Ftaxa_COI)
levels =['Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
df = Geller_COI.copy()
df['Species'] = df['Genus'] + ' ' + df['Species']
df.fillna('no hits', inplace=True)
df['Class']=df['Class'].str.replace('Maxillopoda', 'Hexanauplia')
#df.set_index('#OTU ID', inplace=True)

df[levels]=df[levels].astype(str)
#get rid of white space
for i in levels:
    df[i]=df[i].str.strip()

cols = list(df)
for i in range(len(cols)):
    cols[i] = cols[i].replace('.','_')
df.columns = cols

Geller_COI_taxa = df[levels]

Geller_COI_otu = df[['CP23_1', 'CP23', 'CP23_2', 'GOC2a', 'GOC2b', 'UC10', 'UC1', 'UC12', 'UC13', 
                     'UC14', 'UC15', 'UC2', 'UC3_1', 'UC3', 'UC3_2', 'UC4', 'UC5', 'UC6', 'UC7', 'UC9']]
print(list(df))
#This is rarefied data. Create a percent rarefied data OTU table
df = Geller_COI_otu.copy()
df=df.T
cols = list(df)
df['tot']=df.sum(axis=1)
for i in cols:
    df[i]=df[i]/df['tot'] *100
df.drop('tot', axis=1, inplace=True)
df=df.T
Geller_COI_otu_comp = df.copy()
df

['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
['CP23_1', 'CP23_2', 'CP23', 'GOC2a', 'GOC2b', 'UC10', 'UC12', 'UC13', 'UC14', 'UC15', 'UC1', 'UC2', 'UC3_1', 'UC3_2', 'UC3', 'UC4', 'UC5', 'UC6', 'UC7', 'UC9', 'Sequence', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']


,CP23_1,CP23,CP23_2,GOC2a,GOC2b,UC10,UC1,UC12,UC13,UC14,UC15,UC2,UC3_1,UC3,UC3_2,UC4,UC5,UC6,UC7,UC9
#OTU ID,,,,,,,,,,,,,,,,,,,,
OTU_1,0.212719,0.245167,0.060691,9.074794,7.174748,0.189884,0.000000,1.415120,1.799095,1.382671,4.411208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000601,0.000000
OTU_10,1.819526,1.790682,1.089432,0.116575,0.146620,2.282219,0.000000,6.706046,8.487715,5.893629,14.037027,0.000000,0.000000,0.000000,0.015623,0.001803,0.000000,0.016224,0.000000,0.243965
OTU_100,0.183275,0.233750,0.144817,0.036655,0.000000,0.182674,0.000000,0.007812,0.198297,0.003005,0.044467,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OTU_1000,0.000000,0.000000,0.000000,0.004807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OTU_1001,0.000000,0.000000,0.000000,0.000000,0.002404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OTU_1002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.194692,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OTU_1003,0.000000,0.000000,0.000000,0.096144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OTU_1004,0.000000,0.000000,0.000000,0.009014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
OTU_1005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
#Make compositional rarefied OTU table and taxa table
#18S
# Format Geller Data to get OTU table / taxa table
print(list(Ftaxa_18S))
#levels = list(Ftaxa_18S)
levels =['Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
df = Geller_18S.copy()
df['Species'] = df['Genus'] + ' ' + df['Species']
df.fillna('no hits', inplace=True)
df['Class']=df['Class'].str.replace('Maxillopoda', 'Hexanauplia')
#df.set_index('#OTU ID', inplace=True)

df[levels]=df[levels].astype(str)
#get rid of white space
for i in levels:
    df[i]=df[i].str.strip()

cols = list(df)
for i in range(len(cols)):
    cols[i] = cols[i].replace('.','_')
df.columns = cols

Geller_18S_taxa = df[levels]

Geller_18S_otu = df[['CP23_1', 'CP23', 'CP23_2', 'GOC2a',  'UC10', 'UC1', 'UC12', 'UC13', 
                     'UC14', 'UC15', 'UC2', 'UC3_1', 'UC3', 'UC3_2', 'UC4', 'UC5', 'UC6', 'UC7', 'UC9']]
print(list(df))
#This is rarefied data. Create a percent rarefied data OTU table
df = Geller_18S_otu.copy()
df=df.T
cols = list(df)
df['tot']=df.sum(axis=1)
for i in cols:
    df[i]=df[i]/df['tot'] *100
df.drop('tot', axis=1, inplace=True)
df=df.T
Geller_18S_otu_comp = df.copy()
df

['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
['CP23_1', 'CP23', 'CP23_2', 'GOC2a', 'UC10', 'UC1', 'UC12', 'UC13', 'UC14', 'UC15', 'UC2', 'UC3_1', 'UC3', 'UC3_2', 'UC4', 'UC5', 'UC6', 'UC7', 'UC9', 'Sequence', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species', 'OrganismDescriptionFromGenBank']


,CP23_1,CP23,CP23_2,GOC2a,UC10,UC1,UC12,UC13,UC14,UC15,UC2,UC3_1,UC3,UC3_2,UC4,UC5,UC6,UC7,UC9
#OTU ID,,,,,,,,,,,,,,,,,,,
OTU_1,0.001173,0.001173,0.000000,0.000000,0.068012,0.075048,0.000000,0.000000,0.001173,0.000000,0.024625,0.015244,0.025798,0.008208,0.025798,0.900573,0.005863,0.304882,0.001173
OTU_10,0.024625,0.026970,0.041042,0.001173,0.000000,0.000000,0.260322,0.004690,0.000000,0.007036,0.003518,0.792692,0.722335,3.153180,0.024625,0.021107,0.724680,0.725853,0.000000
OTU_100,0.001173,0.000000,0.000000,0.002345,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002345,0.002345,0.002345,0.290810,0.000000,0.004690,0.001173,0.000000
OTU_101,0.000000,0.000000,0.000000,0.000000,0.000000,0.003518,0.000000,0.000000,0.000000,0.000000,0.001173,0.002345,0.005863,0.001173,0.000000,0.056286,0.000000,0.022280,0.000000
OTU_102,0.000000,0.001173,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003518,0.000000,0.015244,0.018762,0.005863,0.043387,0.017589,0.003518,0.004690,0.000000
OTU_103,5.182988,6.199651,3.483859,0.376412,0.003518,0.000000,0.341233,0.016417,0.300191,3.218846,0.000000,0.002345,0.002345,0.000000,0.001173,0.000000,0.000000,0.000000,0.001173
OTU_104,0.000000,0.000000,0.000000,0.000000,0.001173,0.003518,0.000000,0.000000,0.000000,0.000000,0.002345,0.000000,0.003518,0.002345,0.000000,0.063322,0.000000,0.016417,0.000000
OTU_105,0.000000,0.000000,0.000000,0.001173,0.472567,0.000000,0.016417,0.007036,0.029316,0.003518,0.000000,0.003518,0.004690,0.002345,0.000000,0.090292,0.011726,0.303709,0.000000
OTU_106,0.001173,0.000000,0.001173,0.000000,0.000000,0.144232,0.004690,0.000000,0.000000,0.002345,0.220453,0.140715,0.168858,0.110226,0.007036,0.093810,0.012899,0.077393,0.014071


# Format Metadata

In [9]:
meta_dat = meta_COI.reset_index()
print(list(meta_dat))
#meta_dat['site']= meta_dat.sample_name.str.split('.').str[:-1].str.join('.')
meta_dat['site_Order']=meta_dat['site'].str.replace('GOC2a', '16').str.replace('GOC2b', '17').str.replace('NTC', '0')
meta_dat['site_Order']=meta_dat['site_Order'].str.extract('(\d+)')
meta_dat['site_Order']=meta_dat['site_Order'].astype(int)
meta_dat= meta_dat.loc[meta_dat['sample_type']=='environmental']
meta_dat.sort_values(['site_Order', 'sample_name'], inplace=True)
meta_dat['sample_name']=meta_dat['sample_name'].str.replace('_COI','')
meta_dat['sample_name']=meta_dat['sample_name'].str.replace('.','_')
meta_dat.set_index('sample_name', inplace=True)

meta_dat.head()

['sample_name', 'order', 'tag_sequence', 'tag_sequence_orig', 'primer_sequence_F', 'primer_sequence_R', 'library_tag_combo', 'library', 'sample_type', 'locus', 'tag_number', 'R1', 'R2', 'Treatment', 'depth', 'dec_lat', 'dec_long', 'fluor', 'tmp', 'sal', 'sigma_theta', 'conduct', 'oxy_ml', 'transmiss', 'Description', 'site']


,order,tag_sequence,tag_sequence_orig,primer_sequence_F,primer_sequence_R,library_tag_combo,library,sample_type,locus,tag_number,...,fluor,tmp,sal,sigma_theta,conduct,oxy_ml,transmiss,Description,site,site_Order
sample_name,,,,,,,,,,,,,,,,,,,,,
UC1,1,TTCTAGCT_AA,TTCTAGCT,GGWACWGGWTGAACWGTWTAYCCYCC,TANACYTCNGGRTGNCCRAARAAYCA,N7_TTCTAGCT,N7,environmental,COI,1,...,0.317525,12.57277917,33.31300833,25.17000417,3.879983208,5.731966667,86.42458333,PCNorth,UC1,1
UC2,2,CCTAGAGT_AA,CCTAGAGT,GGWACWGGWTGAACWGTWTAYCCYCC,TANACYTCNGGRTGNCCRAARAAYCA,N8_CCTAGAGT,N8,environmental,COI,2,...,0.327413043,12.72407391,33.28353913,25.11782174,3.890925304,5.685595652,85.7651913,PCNorth,UC2,2
UC3_1,19,GCGTAAGA_AT,GCGTAAGA,GGWACWGGWTGAACWGTWTAYCCYCC,TANACYTCNGGRTGNCCRAARAAYCA,N9_GCGTAAGA,N9,environmental,COI,19,...,0.351265217,12.57266522,33.42426522,25.25619565,3.891625217,5.249904348,85.76833913,PCNorth,UC3,3
UC3_2,17,TTCTAGCT_AT,TTCTAGCT,GGWACWGGWTGAACWGTWTAYCCYCC,TANACYTCNGGRTGNCCRAARAAYCA,N10_TTCTAGCT,N10,environmental,COI,17,...,0.351265217,12.57266522,33.42426522,25.25619565,3.891625217,5.249904348,85.76833913,PCNorth,UC3,3
UC3,3,GCGTAAGA_AA,GCGTAAGA,GGWACWGGWTGAACWGTWTAYCCYCC,TANACYTCNGGRTGNCCRAARAAYCA,N11_GCGTAAGA,N11,environmental,COI,3,...,0.351265217,12.57266522,33.42426522,25.25619565,3.891625217,5.249904348,85.76833913,PCNorth,UC3,3


# MEGAN imports

In [ ]:
#Need a document to import into megan that looks like this:
'''
1,Annelida,5
2,Annelida,5
3,Annelida,5
4,Apicomplexa,5
5,Arthropoda,5
'''

#Then MEGAN will give it Megan names, can export a tree
#Then need to match those megan names to current names (should be similar)
#to add data on number of species etc across COI/18S MLML/MBARI

#Bc of version differences and bc of MLML will get different naming

#Could just export three different trees - get leaf names - and then have trees/lists for each marker/institution!
#Use that to combine and get # of species detected within Family etc


In [31]:
df= Geller_18S_nr.copy()
df.head()

,CP23_1,CP23,CP23_2,GOC2a,GOC2b,UC10,UC1,UC12,UC13,UC14,UC15,UC2,UC3_1,UC3,UC3_2,UC4,UC5,UC6,UC7,UC9
#OTU ID,,,,,,,,,,,,,,,,,,,,
OTU_6,19231,22170,14287,34957,22575,68966,6533,28481,32333,14665,34219,3970,28208,41001,39997,13079,2023,373,10606,1568
OTU_28,16679,18639,13582,1344,1294,5722,25575,26363,29378,14098,32274,21545,20676,23539,31666,16134,14833,35070,25123,24619
OTU_89,10753,11618,4602,22,0,1691,0,1196,223,1298,725,3,0,0,0,3,0,0,0,1
OTU_34,7817,9214,2665,1670,1346,306,27,2595,1881,2881,2781,1955,189,299,68,19,17,1011,36,40
OTU_168,3404,3930,1326,490,244,0,0,1,0,773,100,0,0,0,0,0,0,0,0,0


In [34]:
levels= list(Ftaxa_COI_filt)
MLML_lev = levels[1:]


df=raw_18S_filt.copy()
df.drop('NTC_1', inplace=True, axis=1)
df['Total']=df.sum(axis=1)
df=df.loc[df['Total']>0]
df=pd.concat([df[['Total']], Ftaxa_18S_filt], join='inner', axis=1)
df['18S']=1
df1 = df.copy()
df.head()

df=raw_COI_filt.copy()
df.drop('NTC_2', inplace=True, axis=1)
df['Total']=df.sum(axis=1)
df=df.loc[df['Total']>0]
df=pd.concat([df[['Total']], Ftaxa_COI_filt], join='inner', axis=1)
df['COI']=1
df2=df.copy()
df.head()


df=Geller_COI[MLML_lev].copy()
df['Species'].fillna('no hits', inplace=True)
df['Species_j'] = df['Genus'] + ' ' + df['Species']
#df.loc[df['Species']!='no hits', 'Species']=df['Species_j']
df.fillna('no hits', inplace=True)
df['Class']=df['Class'].str.replace('Maxillopoda', 'Hexanauplia')
df['Gell_COI']=1
df3=df.copy()
#df=df.groupby(MLML_lev).sum()
#df3=df.copy()


df= Geller_18S[MLML_lev].copy()
df['Gell_18S']=1
df['Species'].fillna('no hits', inplace=True)
df['Species'] = df['Genus'] + ' ' + df['Species']
df.fillna('no hits', inplace=True)
df['Class']=df['Class'].str.replace('Maxillopoda', 'Hexanauplia')
#df=df.groupby(MLML_lev).sum()
df4=df.copy()

df=pd.concat([df1,df2, df3, df4], axis=0)
for i in MLML_lev:
    df[i]=df[i].astype(str)
df=df.groupby(MLML_lev).sum()
df=df.sort_index()
df=df.astype(bool).astype(int)
#df['shared']=df['COI']+df['Gell_COI']
#df=df.sort_values('shared', ascending=False)
print(sum(df['COI']))
df.drop('Total', axis=1, inplace=True)
df.reset_index(inplace=True)
#df=df.sort_values(MLML_lev)
#df=df.loc[df['Species'].isin(['s_','no hits', 'not assigned'])== False]
df['Genus']=df['Genus'].fillna('no hits')
df=df.loc[df['Genus'].isin(['g_','no hits', 'not assigned', 'unknown',''])== False]
#Generate seperate text files for each marker/institute
#Go through Genus only?
df_tot=df.copy()
for i in ['18S','COI','Gell_18S','Gell_COI']:
    df=df_tot.loc[df_tot[i]>0]
    df=df[['Genus']]
    df['Score']=5
    df.to_csv('/Users/kpitz/Projects/Graphlan_Tree/'+i+'_tree_input_genus_100719.csv')
#all
df_tot=df_tot[['Genus']]    
df_tot['Score']=5

df_tot.to_csv('/Users/kpitz/Projects/Graphlan_Tree/All_tree_input_genus_100719.csv')
df_tot


/Users/kpitz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



670


,Genus,Score
0,Vanadis,5
1,Archinome,5
2,Chaetoparia,5
3,Eumida,5
4,Eumida,5
5,Phyllodoce,5
6,Tomopteris,5
7,,5
8,Gunnarea,5
9,Laonice,5


In [ ]:
#Imported into MEGAN, exported Tree file (.tre) from Total -> changed to Nexus in Figtree.
#Exported taxonomy csv from other files

In [15]:
#Need to get genus annotations back out from MEGAN
#*-ex.txt files
#/Users/kpitz/Projects/Graphlan_Tree/Gell_18S_tree_input_genus-ex.txt
#Want to create rings through Graphlan
#Automate annotation file generation
#For each annotation, mark with # taxa in each genus
dfs=[]
names= []
resultsFilename = '/Users/kpitz/Projects/Graphlan_Tree/GOC_Annot_MB_MLML_102419.txt'
resultsfile = open(resultsFilename, "w")

resultsfile.write('title')
resultsfile.write('\t')
#resultsfile.write(filename.split('/')[-1].split('_')[0])
resultsfile.write('GOC MLML MBARI Tree')
resultsfile.write('\n')
resultsfile.write('title_font_size')
resultsfile.write('\t')
resultsfile.write('10\n')
#resultsfile.write('total_plotted_degrees\t360\n')
#resultsfile.write('annotation_background_alpha\t0.1')
#resultsfile.write('\n')
#resultsfile.write('clade_marker_size\t0.001\n')

resultsfile.write('total_plotted_degrees\t340\n')
resultsfile.write('start_rotation\t270\n')
resultsfile.write('annotation_background_alpha\t0.1')
resultsfile.write('\n')
resultsfile.write('clade_marker_size\t1\n')
resultsfile.write('clade_marker_edge_width\t0.0001\n')

count=1
#names = ['Prayidae', 'Alviniconcha', 'Corymorphidae', 'Eudendriidae', 'Helotiales', 'unclassified']  #caused an issue in plotting
#for i,color in zip(['18S','COI','Gell_18S','Gell_COI'],['#0000FF','#FFA500','#FF0000','#800000']):
for i,color in zip(['18S','Gell_18S','COI','Gell_COI'],['#0000FF','#FFA500','#FF0000','#800000']): 
    print(i)
    #for i,color in zip(['18S','COI','Gell_COI'],['#0000FF','#FFA500','#800000']):    
    df=pd.read_csv('/Users/kpitz/Projects/Graphlan_Tree/'+i+'_tree_input_genus_100719-ex.txt', names=['Name',i])
    #df[i]=1
    height='0.2'
    dfs.append(df)
    #For each ring
    str_count = str(count)
    print(str_count)
    #resultsfile.write('ring_internal_separator_thickness\t'+str(count)+'\t0.5\n')
    #resultsfile.write('ring_width\t'+str(count)+'\t0.8\n')
    #resultsfile.write('ring_height\t'+str(count)+'\t0.2\n')
    #resultsfile.write('ring_label\t'+str(count)+'\t'+i+'\n')
    #resultsfile.write('ring_label_color\t'+str(count)+'\t'+color+'\n')
    resultsfile.write('ring_internal_separator_thickness\t'+str(count)+'\t0.5\n')
    resultsfile.write('ring_width\t'+str(count)+'\t0.9\n')
    #resultsfile.write('ring_height\t'+str(count)+'\t0.4\n')
    resultsfile.write('ring_height\t'+str(count)+'\t0.9\n')
    resultsfile.write('ring_label\t'+str(count)+'\t'+i+'\n')
    resultsfile.write('ring_label_color\t'+str(count)+'\t'+color+'\n')
    resultsfile.write('ring_label_font_size\t'+str(count)+'\t'+'12'+'\n')
    resultsfile.write('ignore_branch_len\t'+'1'+'\n')
    
    df.drop_duplicates('Name',inplace=True)
    gen = df['Name'].tolist()
    #gen = list(set(gen))
    for x in gen:
        genera =str(x)
        #genera = genera.split(' ')[0]
        genera = genera.replace(' ','_')
        if genera in names:
            print(genera)
            continue
        #if genera in Fam_names:
        #    print(genera)
        #    continue
        #if '\t' in genera:
        #    print(genera)
        resultsfile.write(genera+'\tring_color\t'+str_count+'\t'+color+'\n')
        #resultsfile.write(genera+'\tring_height\t'+str_count+'\t'+height+'\n')
    #if count >=3:
    #    break
    #print(df.head())
    count+=1

df=pd.concat(dfs, axis=0)
df=df.groupby('Name').sum()
#345 genera

#Also want to label by Phylum? - for now don't worry about it, could merge with taxa_table

#Names with issues
#for genera in names:
#    resultsfile.write(genera+'\tring_color\t'+'3'+'\t#FF0000\n')

#[&label="Euphausiacea"]  replace with Euphausiacea in nexus file

resultsfile.close()
print ('Done with Analysis')  

df

18S
1
Gell_18S
2
COI
3
Gell_COI
4
Done with Analysis


/Users/kpitz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:81: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,18S,COI,Gell_18S,Gell_COI
Name,,,,
Abylidae,0.0,1.0,0.0,1.0
Acartiidae,2.0,3.0,1.0,3.0
Actinostolidae,0.0,1.0,0.0,1.0
Aeginidae,1.0,0.0,1.0,0.0
Aequoreidae,0.0,0.0,0.0,1.0
Aetideidae,0.0,4.0,0.0,2.0
Agalmatidae,3.0,7.0,0.0,4.0
Alciopidae,0.0,0.0,1.0,0.0
Alviniconcha,0.0,0.0,1.0,0.0


In [ ]:
(graphlan) mbari2030:Graphlan_Tree kpitz$ graphlan_annotate.py --annot GOC_Annot_MB_MLML_102419.txt All_tree_input_genus_100719_edited.nexus new_Gell_MB_comp.xml
(graphlan) mbari2030:Graphlan_Tree kpitz$ graphlan.py new_Gell_MB_comp.xml test_MBMLML_102419.png --dpi 300 --size 6
    
    

In [ ]:
'''Arthropoda	annotation	Arthropoda
Mollusca	annotation	Mollusca
Cnidaria	annotation	Cnidaria
Annelida	annotation	Annelida
Actinopteri	annotation	Actinopteri
Echinodermata	annotation	Echinodermata
Dinophyceae	annotation	Dinophyceae
'''

In [30]:
levels[:6]

['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus']

In [47]:
tax_lev = 'Genus'

# Diversity layer - based on Banzai
#Number of taxonomic annotation by Phyla / Class / Family by Region

levels = list(Ftaxa_COI_filt)

df=raw_COI_filt.copy()
df=df.T
df=pd.concat([df,meta_dat['Description']], axis=1, join='inner')
df=df.loc[df['Description']!='0']
print(len(df.index))
#20samps
df = df.groupby('Description').sum()
df=df.astype(bool).astype(int)
df=pd.concat([df.T, Ftaxa_COI_filt], axis=1, join='inner')
df=df.loc[df['Kingdom']!='not assigned']
df=df.loc[df['Kingdom']!='no hits']
#df=df.loc[df['Class']!='c_']
#df=df.loc[df['Class']!='unknown']
df=df.loc[df['Genus'].isin(['g_','no hits', 'not assigned', 'unknown',''])== False]
df=df.loc[df[tax_lev].isin(['g_','no hits', 'not assigned', 'unknown',''])== False]
df=df.groupby(levels[:6]).sum() #now have number of genera
#get num taxa
df=df.astype(bool).astype(int)

df.reset_index(inplace=True)

#df=df.groupby(['Kingdom','Phylum','Class','Order', 'Family', 'Genus']).sum() 
df=df.groupby(levels[:6]).sum() 
#df=df.astype(bool).astype(int)
taxa_class_COI=df.copy()
df.reset_index(inplace=True)
df_tot=df.copy()
for i in ['EUNorth','EUSouth','PCNorth']:
    df=df_tot.loc[df_tot[i]>0]
    df=df[[tax_lev, i]]
    #df['Score']=5
    df.to_csv('/Users/kpitz/Projects/Graphlan_Tree/'+i+'_tree_input_genus_100319_Description.csv')
#all
df_tot['total'] = df_tot.sum(axis=1)
df=df_tot.copy()
df=df[[tax_lev, 'total']]    
#df_tot['Score']=5
#remove Family names after running scripts below
#df_tot = df_tot.loc[df_tot['Genus'].isin(Fam_names)==False]

#Decided to manually remove the following, might have spaces or something
#Alviniconcha
#Corymorphidae
#Eudendriidae
#Prayidae

df.to_csv('/Users/kpitz/Projects/Graphlan_Tree/All_tree_input_genus_100319_Description.csv')
df.head()


20


,Genus,total
0,Chloeia,2
1,Phyllodoce <polychaete>,2
2,Phragmatopoma,1
3,Chone,1
4,Phyllochaetopterus,3


In [57]:
'''#Make annotation input file for Graphlan
#Need to get annotations back out from MEGAN - sometimes don't match (USEARCH)
#export all leaves within MEGAN (taxonomy) - csv format
#*-ex.txt files
file = '/Users/kpitz/Projects/Graphlan_Tree/All_tree_input_genus_100319_Description-ex.txt'
df=pd.read_csv(file, names=['Megan','Total'])
df.set_index('Megan', inplace=True)
df1 = df_tot.copy()
df1['Genus']=df1['Genus'].str.split(' ').str[0] #some megan annots had two annotations (e.g. <hydrozoan> )
df1.set_index('Genus', inplace=True)
#df=pd.concat([df,df_tot.set_index('Genus') ], axis=1)
df=pd.concat([df,df1 ], axis=1)
#problem_names
#Bassia
#Phyllodoce 
df2=df.loc[df['Kingdom'].isna()==True] #check for unmerged names
print(df2)
df2=df.loc[df['Total'].isna()==True] #check for unmerged names
print(df2)
df'''


Empty DataFrame
Columns: [Total, Kingdom, Phylum, Class, Order, Family, EUNorth, EUSouth, PCNorth, total]
Index: []
      Total    Kingdom    Phylum       Class        Order        Family  \
Clio    NaN  Eukaryota  Mollusca  Gastropoda  Thecosomata  Cavoliniidae   

      EUNorth  EUSouth  PCNorth  total  
Clio        1        1        0      2  


/Users/kpitz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Total,Kingdom,Phylum,Class,Order,Family,EUNorth,EUSouth,PCNorth,total
Abylopsis,1.0,Eukaryota,Cnidaria,Hydrozoa,Siphonophorae,Abylidae,0,1,0,1
Acanthodoris,1.0,Eukaryota,Mollusca,Gastropoda,Nudibranchia,Onchidorididae,1,0,0,1
Acartia,1.0,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Acartiidae,1,1,1,3
Acrocalanus,1.0,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Paracalanidae,0,1,0,1
Agalma,1.0,Eukaryota,Cnidaria,Hydrozoa,Siphonophorae,Agalmatidae,1,1,1,3
Alexandrium,1.0,Eukaryota,unknown,Dinophyceae,Gonyaulacales,Gonyaulacaceae,1,1,1,3
Allocentrotus,1.0,Eukaryota,Echinodermata,Echinoidea,Echinoida,Strongylocentrotidae,0,0,1,1
Amphicaryon,1.0,Eukaryota,Cnidaria,Hydrozoa,Siphonophorae,Prayidae,1,1,1,3
Amphipholis,1.0,Eukaryota,Echinodermata,Ophiuroidea,Ophiurida,Amphiuridae,1,0,1,2
Amphissa,1.0,Eukaryota,Mollusca,Gastropoda,Neogastropoda,Columbellidae,1,0,0,1


In [84]:
#Need to get genus annotations back out from MEGAN
#export all leaves within MEGAN (taxonomy) - csv format
#*-ex.txt files

#/Users/kpitz/Projects/Graphlan_Tree/Gell_18S_tree_input_genus-ex.txt
#Want to create rings through Graphlan
#Automate annotation file generation
#For each annotation, mark with # genera in each class
dfs=[]

resultsFilename = '/Users/kpitz/Projects/Graphlan_Tree/GOC_Annot_Region_100719.txt'
resultsfile = open(resultsFilename, "w")

resultsfile.write('title')
resultsfile.write('\t')
#resultsfile.write(filename.split('/')[-1].split('_')[0])
resultsfile.write('GOC COI Region Num Genus Tree')
resultsfile.write('\n')
resultsfile.write('title_font_size')
resultsfile.write('\t')
resultsfile.write('10\n')
resultsfile.write('total_plotted_degrees\t340\n')
resultsfile.write('start_rotation\t270\n')
resultsfile.write('annotation_background_alpha\t0.1')
resultsfile.write('\n')
resultsfile.write('clade_marker_size\t1\n')
resultsfile.write('clade_marker_edge_width\t0.0001\n')

count=1
names = ['Prayidae']  #caused an issue in plotting
#for i,color in zip(['18S','COI','Gell_18S','Gell_COI'],['#0000FF','#FFA500','#FF0000','#800000']):
for i,color in zip(['PCNorth','EUNorth','EUSouth'],['#0000FF','#FFA500','#FF0000']):    
    #for i,color in zip(['18S','COI','Gell_COI'],['#0000FF','#FFA500','#800000']):    
    df=pd.read_csv('/Users/kpitz/Projects/Graphlan_Tree/'+i+'_tree_input_genus_100319_Description-ex.txt', names=['Name',i])
    #df[i]=1
    height='0.2'
    dfs.append(df)
    #For each ring
    str_count = str(count)
    print(str_count)
    resultsfile.write('ring_internal_separator_thickness\t'+str(count)+'\t0.5\n')
    resultsfile.write('ring_width\t'+str(count)+'\t0.9\n')
    resultsfile.write('ring_height\t'+str(count)+'\t0.4\n')
    resultsfile.write('ring_label\t'+str(count)+'\t'+i+'\n')
    resultsfile.write('ring_label_color\t'+str(count)+'\t'+color+'\n')
    resultsfile.write('ignore_branch_len\t'+'1'+'\n')
    
    df.drop_duplicates('Name',inplace=True)
    gen = df['Name'].tolist()
    #gen = list(set(gen))
    for x in gen:
        genera =str(x)
        if genera in names:
            print(genera)
            continue
        #if genera in Fam_names:
        #    print(genera)
            continue
        #if '\t' in genera:
        #    print(genera)
        
        resultsfile.write(genera+'\tring_color\t'+str_count+'\t'+color+'\n')
        #resultsfile.write(genera+'\tclade_marker_color\t'+color+'\n')
        #resultsfile.write(genera+'\tring_height\t'+str_count+'\t'+height+'\n')
    #if count >=3:
    #    break
    print(df.head())
    count+=1

df=pd.concat(dfs, axis=0)
df=df.groupby('Name').sum()

#label Genera that are only present in one region?
df['Total']=df.sum(axis=1)
df_left = df.loc[df['Total']!=1]
df=df.loc[df['Total']==1]
df_tot=df.copy()
for i,color in zip(['PCNorth','EUNorth','EUSouth'],['#0000FF','#FFA500','#FF0000']):    
    df=df_tot.loc[df_tot[i]>0]
    df.reset_index(inplace=True)
    names = df['Name'].tolist()
    for genera in names:
        resultsfile.write(genera+'\tclade_marker_color\t'+color+'\n')
        resultsfile.write(genera+'\tclade_marker_size\t'+'9'+'\n')
    
#color grey the muliple ones
df=df_left.reset_index()
names = df['Name'].tolist()
for genera in names:
    resultsfile.write(genera+'\tclade_marker_color\t'+'#c6cad1'+'\n')
    resultsfile.write(genera+'\tclade_marker_size\t'+'4'+'\n')


#345 genera

#Also want to label by Phylum? - for now don't worry about it, could merge with taxa_table

#Names with issues
#for genera in names:
#    resultsfile.write(genera+'\tring_color\t'+'3'+'\t#FF0000\n')

#[&label="Euphausiacea"]  replace with Euphausiacea in nexus file

resultsfile.close()
print ('Done with Analysis')  

df

1
           Name  PCNorth
0   Alexandrium      1.0
1   Gymnodinium      1.0
2  Woloszynskia      1.0
3  Prorocentrum      1.0
4    Pyrocystis      1.0
2
           Name  EUNorth
0   Alexandrium      1.0
1   Gymnodinium      1.0
2  Lepidodinium      1.0
3  Prorocentrum      1.0
4    Pyrocystis      1.0
3
           Name  EUSouth
0   Alexandrium      1.0
1   Gymnodinium      1.0
2  Lepidodinium      1.0
3  Prorocentrum      1.0
4    Pyrocystis      1.0
Done with Analysis


/Users/kpitz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:70: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,Name,EUNorth,EUSouth,PCNorth,Total
0,Acartia,1.0,1.0,1.0,3.0
1,Agalma,1.0,1.0,1.0,3.0
2,Alexandrium,1.0,1.0,1.0,3.0
3,Amphicaryon,1.0,1.0,1.0,3.0
4,Amphipholis,1.0,0.0,1.0,2.0
5,Aplysia,1.0,1.0,0.0,2.0
6,Astropecten,1.0,0.0,1.0,2.0
7,Athorybia,0.0,1.0,1.0,2.0
8,Balanus,0.0,1.0,1.0,2.0
9,Bassia,1.0,1.0,0.0,2.0


(graphlan) mbari2030:Graphlan_Tree kpitz$ graphlan_annotate.py --annot GOC_Annot_Region_100719.txt All_tree_input_genus_100319_Description_extreme.nexus new_cutadapt_Region.xml
(graphlan) mbari2030:Graphlan_Tree kpitz$ graphlan.py new_cutadapt_Region.xml test_Region_label.png --dpi 300 --size 6
(graphlan) mbari2030:Graphlan_Tree kpitz$ graphlan.py new_cutadapt_Region.xml test_Region_label.svg --dpi 300 --size 6

In [ ]:
#Phylum labels in own ring
'''
ring_internal_separator_thickness	4	0.5
ring_width	4	0.9
ring_height	4	0.4
ring_label	4	Phylum
ring_label_color	4	#0000FF
Arthropoda	ring_color	4	lightgrey
Mollusca	ring_color	4	grey
Annelida	ring_color	4	grey
Actinopteri	ring_color	4	grey
Echinodermata	ring_color	4	grey
Cnidaria	ring_color	4	grey
'''



#Arthropoda	annotation	Arthropoda
#Mollusca	annotation	Mollusca
#Cnidaria	annotation	Cnidaria
#Annelida	annotation	Annelida
#Actinopteri	annotation	Actinopteri
#Echinodermata	annotation	Echinodermata

In [ ]:
ring_internal_separator_thickness	4	0.5
ring_width	4	0.9
ring_height	4	0.4
ring_label	4	Phylum
ring_label_color	4	#0000FF
Arthropoda	ring_color	4	lightgrey
Mollusca	ring_color	4	grey
Annelida	ring_color	4	grey
Actinopteri	ring_color	4	grey
Echinodermata	ring_color	4	grey
Cnidaria	ring_color	4	grey

Arthropoda	annotation	Arthropoda
Mollusca	annotation	Mollusca
Cnidaria	annotation	Cnidaria
Annelida	annotation	Annelida
Actinopteri	annotation	Actinopteri
Echinodermata	annotation	Echinodermata

In [ ]:
# Add to Annot File
'''Arthropoda	annotation	Arthropoda
#Mollusca	clade_marker_color	#1f78b4
Mollusca	annotation	Mollusca
#Dinophyceae	clade_marker_color	#b2df8a
Dinophyceae	annotation	Dinophyceae
#Cnidaria	clade_marker_color	#33a02c
Cnidaria	annotation	Cnidaria
#Annelida	clade_marker_color	#fb9a99
Annelida	annotation	Annelida
#Actinopteri	clade_marker_color	#e31a1c
Actinopteri	annotation	Actinopteri
#Echinodermata	clade_marker_color	#fdbf6f
Echinodermata	annotation	Echinodermata
#Tunicata	clade_marker_color	#ff7f00
Tunicata	annotation	Tunicata
#Stramenopiles	clade_marker_color	#cab2d6
Stramenopiles	annotation	Stramenopiles
#Viridiplantae	clade_marker_color	#6a3d9a
Viridiplantae	annotation	Viridiplantae
#Ctenophora	clade_marker_color	#ffff99
Ctenophora	annotation	Ctenophora
#Chaetognatha	clade_marker_color	#b15928
Chaetognatha	annotation	Chaetognatha'''

graphlan_annotate.py --annot GOC_Annot_Region_100319.txt All_tree_input_genus_100319_Description.nexus new_cutadapt_Region.xml

#source activate graphlan
#in Figtree converted .tre file from MEGAN into nexus file (w/ annotations)
#Then in order to annotate phylums, need to find and replace '"]' with '' and 

graphlan.py new_cutadapt_Region.xml test_Region.png --dpi 300 --size 10

In [ ]:
#Need to get genus annotations back out from MEGAN
#export all leaves within MEGAN (taxonomy) - csv format
#*-ex.txt files

#/Users/kpitz/Projects/Graphlan_Tree/Gell_18S_tree_input_genus-ex.txt
#Want to create rings through Graphlan
#Automate annotation file generation
#For each annotation, mark with # genera in each class
dfs=[]

resultsFilename = '/Users/kpitz/Projects/Graphlan_Tree/GOC_Annot_Region_100319.txt'
resultsfile = open(resultsFilename, "w")

resultsfile.write('title')
resultsfile.write('\t')
#resultsfile.write(filename.split('/')[-1].split('_')[0])
resultsfile.write('GOC COI Region Num Genus Tree')
resultsfile.write('\n')
resultsfile.write('title_font_size')
resultsfile.write('\t')
resultsfile.write('10\n')
resultsfile.write('total_plotted_degrees\t340\n')
resultsfile.write('start_rotation\t270\n')
resultsfile.write('annotation_background_alpha\t0.1')
resultsfile.write('\n')
resultsfile.write('clade_marker_size\t5\n')
resultsfile.write('clade_marker_edge_width\t0.0001\n')

count=1
names = ['Prayidae']  #caused an issue in plotting
#for i,color in zip(['18S','COI','Gell_18S','Gell_COI'],['#0000FF','#FFA500','#FF0000','#800000']):
for i,color in zip(['PCNorth','EUNorth','EUSouth'],['#0000FF','#FFA500','#FF0000']):    
    #for i,color in zip(['18S','COI','Gell_COI'],['#0000FF','#FFA500','#800000']):    
    df=pd.read_csv('/Users/kpitz/Projects/Graphlan_Tree/'+i+'_tree_input_genus_100319_Description-ex.txt', names=['Name',i])
    #df[i]=1
    height='0.2'
    dfs.append(df)
    #For each ring
    str_count = str(count)
    print(str_count)
    resultsfile.write('ring_internal_separator_thickness\t'+str(count)+'\t0.5\n')
    resultsfile.write('ring_width\t'+str(count)+'\t0.8\n')
    resultsfile.write('ring_height\t'+str(count)+'\t0.8\n')
    resultsfile.write('ring_label\t'+str(count)+'\t'+i+'\n')
    resultsfile.write('ring_label_color\t'+str(count)+'\t'+color+'\n')
    
    df.drop_duplicates('Name',inplace=True)
    gen = df['Name'].tolist()
    #gen = list(set(gen))
    for x in gen:
        genera =str(x)
        if genera in names:
            print(genera)
            continue
        #if genera in Fam_names:
        #    print(genera)
            continue
        #if '\t' in genera:
        #    print(genera)
        
        resultsfile.write(genera+'\tring_color\t'+str_count+'\t'+color+'\n')
        #resultsfile.write(genera+'\tclade_marker_color\t'+color+'\n')
        #resultsfile.write(genera+'\tring_height\t'+str_count+'\t'+height+'\n')
    #if count >=3:
    #    break
    print(df.head())
    count+=1

df=pd.concat(dfs, axis=0)
df=df.groupby('Name').sum()

#label Genera that are only present in one region?
df['Total']=df.sum(axis=1)
df_left = df.loc[df['Total']!=1]
df=df.loc[df['Total']==1]
df_tot=df.copy()
for i,color in zip(['PCNorth','EUNorth','EUSouth'],['#0000FF','#FFA500','#FF0000']):    
    df=df_tot.loc[df_tot[i]>0]
    df.reset_index(inplace=True)
    names = df['Name'].tolist()
    for genera in names:
        resultsfile.write(genera+'\tclade_marker_color\t'+color+'\n')
    
#color grey the muliple ones
df=df_left.reset_index()
names = df['Name'].tolist()
for genera in names:
    resultsfile.write(genera+'\tclade_marker_color\t'+'#c6cad1'+'\n')



#345 genera

#Also want to label by Phylum? - for now don't worry about it, could merge with taxa_table

#Names with issues
#for genera in names:
#    resultsfile.write(genera+'\tring_color\t'+'3'+'\t#FF0000\n')

#[&label="Euphausiacea"]  replace with Euphausiacea in nexus file

resultsfile.close()
print ('Done with Analysis')  

df

In [82]:
#Try making another ring alternating color by Phylum
#Need to get genus annotations back out from MEGAN
#export all leaves within MEGAN (taxonomy) - csv format
#*-ex.txt files

#/Users/kpitz/Projects/Graphlan_Tree/Gell_18S_tree_input_genus-ex.txt
#Want to create rings through Graphlan
#Automate annotation file generation
#For each annotation, mark with # genera in each class
dfs=[]

#resultsFilename = '/Users/kpitz/Projects/Graphlan_Tree/GOC_Annot_Region_100319.txt'


count=1
names = ['Prayidae']  #caused an issue in plotting
#for i,color in zip(['18S','COI','Gell_18S','Gell_COI'],['#0000FF','#FFA500','#FF0000','#800000']):
for i,color in zip(['PCNorth','EUNorth','EUSouth'],['#0000FF','#FFA500','#FF0000']):    
    #for i,color in zip(['18S','COI','Gell_COI'],['#0000FF','#FFA500','#800000']):    
    df=pd.read_csv('/Users/kpitz/Projects/Graphlan_Tree/'+i+'_tree_input_genus_100319_Description-ex.txt', names=['Name',i])
    #df[i]=1
    height='0.2'
    dfs.append(df)
    #For each ring
    str_count = str(count)
    print(str_count)

    df.drop_duplicates('Name',inplace=True)
    gen = df['Name'].tolist()
    #gen = list(set(gen))
    for x in gen:
        genera =str(x)
        if genera in names:
            print(genera)
            continue
        #if genera in Fam_names:
        #    print(genera)
            continue
        #if '\t' in genera:
        #    print(genera)
        
        #resultsfile.write(genera+'\tring_color\t'+str_count+'\t'+color+'\n')
        #resultsfile.write(genera+'\tclade_marker_color\t'+color+'\n')
        #resultsfile.write(genera+'\tring_height\t'+str_count+'\t'+height+'\n')
    #if count >=3:
    #    break
    print(df.head())
    count+=1

df=pd.concat(dfs, axis=0)
df=df.groupby('Name').sum()

#label Genera that are only present in one region?
df['Total']=df.sum(axis=1)
df_left = df.loc[df['Total']!=1]
df=df.loc[df['Total']==1]
df_tot=df.copy()
for i,color in zip(['PCNorth','EUNorth','EUSouth'],['#0000FF','#FFA500','#FF0000']):    
    df=df_tot.loc[df_tot[i]>0]
    df.reset_index(inplace=True)
    names = df['Name'].tolist()
    #for genera in names:
        #resultsfile.write(genera+'\tclade_marker_color\t'+color+'\n')
    
#color grey the muliple ones
df=df_left.reset_index()
names = df['Name'].tolist()
#for genera in names:
    #resultsfile.write(genera+'\tclade_marker_color\t'+'#c6cad1'+'\n')



#345 genera

#Also want to label by Phylum? - for now don't worry about it, could merge with taxa_table

#Names with issues
#for genera in names:
#    resultsfile.write(genera+'\tring_color\t'+'3'+'\t#FF0000\n')

#[&label="Euphausiacea"]  replace with Euphausiacea in nexus file

#resultsfile.close()
print ('Done with Analysis')  
df.set_index('Name', inplace=True)
df1=Ftaxa_COI_filt.drop_duplicates('Genus')
df1['Genus']= df1['Genus'].str.split(' ').str[0]
df1 = df1.set_index('Genus')

df=df.join(df1)
df=df.sort_values('Phylum')
print('Num Unique Phyla',len(df['Phylum'].unique()))
print(df['Phylum'].unique())
phyla_list =[]
#df=df.loc[df['Phylum'].isna()==True]
df

1
           Name  PCNorth
0   Alexandrium      1.0
1   Gymnodinium      1.0
2  Woloszynskia      1.0
3  Prorocentrum      1.0
4    Pyrocystis      1.0
2
           Name  EUNorth
0   Alexandrium      1.0
1   Gymnodinium      1.0
2  Lepidodinium      1.0
3  Prorocentrum      1.0
4    Pyrocystis      1.0
3
           Name  EUSouth
0   Alexandrium      1.0
1   Gymnodinium      1.0
2  Lepidodinium      1.0
3  Prorocentrum      1.0
4    Pyrocystis      1.0
Done with Analysis
12


/Users/kpitz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/Users/kpitz/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,EUNorth,EUSouth,PCNorth,Total,Kingdom,Phylum,Class,Order,Family,Species
Name,,,,,,,,,,
Chloeia,1.0,1.0,0.0,2.0,Eukaryota,Annelida,Polychaeta,Eunicida,Amphinomidae,s_
Phyllochaetopterus,1.0,1.0,1.0,3.0,Eukaryota,Annelida,Polychaeta,Spionida,Chaetopteridae,Phyllochaetopterus prolifica
Phyllodoce,1.0,0.0,1.0,2.0,Eukaryota,Annelida,Polychaeta,Phyllodocida,Phyllodocidae,Phyllodoce medipapillata
Paraconchoecia,1.0,0.0,1.0,2.0,Eukaryota,Arthropoda,Ostracoda,Halocyprida,Halocyprididae,Paraconchoecia allotherium
Paracalanus,1.0,1.0,1.0,3.0,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Paracalanidae,s_
Oncaea,1.0,1.0,1.0,3.0,Eukaryota,Arthropoda,Hexanauplia,Poecilostomatoida,Oncaeidae,Oncaea scottodicarloi
Oithona,1.0,1.0,1.0,3.0,Eukaryota,Arthropoda,Hexanauplia,Cyclopoida,Oithonidae,s_
Pareucalanus,1.0,1.0,0.0,2.0,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Eucalanidae,s_
Acartia,1.0,1.0,1.0,3.0,Eukaryota,Arthropoda,Hexanauplia,Calanoida,Acartiidae,s_
